In [16]:
#Import Necessary Packages
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
#Load lasso model
filename='lasso_model.sav'
lasso_model =pickle.load(open(filename, 'rb'))

In [3]:
#Load 2019 Free Agents
df_2019 = pd.read_csv('predict.csv')

#Fill na free throw% and three point% with 0
df_2019 = df_2019.fillna(0)
X_new = df_2019[['PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'VORP', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'ORtg', 'DRtg']]

#Create predictions for 2019 free agents and export into csv ussing lasso model
y_pred_new = lasso_model.predict(X_new)
pred_df = pd.DataFrame(data = y_pred_new.flatten())
lasso_df = pd.concat([df_2019['Player'], pred_df], axis =1)
lasso_df.to_csv('lasso_pred.csv')
lasso_df.sort_values(by=0, ascending=False).head()

,Player,0
7,Kyrie Irving\irvinky01,36.450553
33,Kemba Walker\walkeke02,36.425498
27,Nikola Vucevic\vucevni01,33.986964
0,Kevin Durant\duranke01,33.383368
4,Kawhi Leonard\leonaka01,32.841971


In [10]:
true_df = pd.read_csv('2019_true.csv', encoding = "ISO-8859-1")
true_df.head()

,Rk,Player,Pos,Age,Type,OTm,2018-19 Stats,WS,NTm,Terms,Notes
0,1,Kevin Durant\duranke01,F-G,30-313,UFA,GSW,26.0 Pts 6.4 Reb 5.9 Ast,11.5,BRK,Signed a 4-yr $164.4MM contract with GSW as pa...,NaN
1,2,Nikola Vucevic\vucevni01,C,28-288,UFA,ORL,20.8 Pts 12.0 Reb 3.8 Ast,10.1,ORL,Signed 4-yr $100M contract,NaN
2,3,Kawhi Leonard\leonaka01,F,28-040,UFA,TOR,26.6 Pts 7.3 Reb 3.3 Ast,9.5,LAC,Signed 3-yr $103.13M contract,NaN
3,4,Kyrie Irving\irvinky01,G,27-138,UFA,BOS,23.8 Pts 5.0 Reb 6.9 Ast,9.1,BRK,Signed 4-yr $140.7M contract,NaN
4,5,Jimmy Butler\butleji01,F-G,29-328,UFA,PHI,18.7 Pts 5.3 Reb 4.0 Ast,7.9,MIA,Signed a 4-yr $140.7MM contract with PHI as pa...,NaN


In [12]:
#Extract salary total and years from terms column
true_df['Salary_Total'] = true_df['Terms'].apply(lambda st: st[st.find('$')+1:st.find('M')])
true_df['Years'] = true_df['Terms'].apply(lambda st: st.split('-')[0][-1])

#Convert salary total and years to floats
true_df['Salary_Total'] = true_df['Salary_Total'].astype(float)
true_df['Years'] = true_df['Years'].astype(float)

#Create Yearly Average column 
true_df['Year_Avg'] = true_df['Salary_Total']/true_df['Years']

true_df.head()

,Rk,Player,Pos,Age,Type,OTm,2018-19 Stats,WS,NTm,Terms,Notes,Salary_Total,Years,Year_Avg
0,1,Kevin Durant\duranke01,F-G,30-313,UFA,GSW,26.0 Pts 6.4 Reb 5.9 Ast,11.5,BRK,Signed a 4-yr $164.4MM contract with GSW as pa...,NaN,164.40,4.0,41.100000
1,2,Nikola Vucevic\vucevni01,C,28-288,UFA,ORL,20.8 Pts 12.0 Reb 3.8 Ast,10.1,ORL,Signed 4-yr $100M contract,NaN,100.00,4.0,25.000000
2,3,Kawhi Leonard\leonaka01,F,28-040,UFA,TOR,26.6 Pts 7.3 Reb 3.3 Ast,9.5,LAC,Signed 3-yr $103.13M contract,NaN,103.13,3.0,34.376667
3,4,Kyrie Irving\irvinky01,G,27-138,UFA,BOS,23.8 Pts 5.0 Reb 6.9 Ast,9.1,BRK,Signed 4-yr $140.7M contract,NaN,140.70,4.0,35.175000
4,5,Jimmy Butler\butleji01,F-G,29-328,UFA,PHI,18.7 Pts 5.3 Reb 4.0 Ast,7.9,MIA,Signed a 4-yr $140.7MM contract with PHI as pa...,NaN,140.70,4.0,35.175000


In [19]:
join_df = pd.merge(true_df, lasso_df, on='Player', how='inner')
join_df.head()

,Rk,Player,Pos,Age,Type,OTm,2018-19 Stats,WS,NTm,Terms,Notes,Salary_Total,Years,Year_Avg,0
0,1,Kevin Durant\duranke01,F-G,30-313,UFA,GSW,26.0 Pts 6.4 Reb 5.9 Ast,11.5,BRK,Signed a 4-yr $164.4MM contract with GSW as pa...,NaN,164.40,4.0,41.100000,33.383368
1,2,Nikola Vucevic\vucevni01,C,28-288,UFA,ORL,20.8 Pts 12.0 Reb 3.8 Ast,10.1,ORL,Signed 4-yr $100M contract,NaN,100.00,4.0,25.000000,33.986964
2,3,Kawhi Leonard\leonaka01,F,28-040,UFA,TOR,26.6 Pts 7.3 Reb 3.3 Ast,9.5,LAC,Signed 3-yr $103.13M contract,NaN,103.13,3.0,34.376667,32.841971
3,4,Kyrie Irving\irvinky01,G,27-138,UFA,BOS,23.8 Pts 5.0 Reb 6.9 Ast,9.1,BRK,Signed 4-yr $140.7M contract,NaN,140.70,4.0,35.175000,36.450553
4,5,Jimmy Butler\butleji01,F-G,29-328,UFA,PHI,18.7 Pts 5.3 Reb 4.0 Ast,7.9,MIA,Signed a 4-yr $140.7MM contract with PHI as pa...,NaN,140.70,4.0,35.175000,31.448680


In [20]:
y_pred = join_df[0]
y_true = join_df['Year_Avg']
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 5.595945


In [21]:
#Load ridge model
filename='ridge_model.sav'
ridge_model =pickle.load(open(filename, 'rb'))

In [24]:
#Create predictions for 2019 free agents and export into csv ussing ridge model
y_pred_new = ridge_model.predict(X_new)
pred_df = pd.DataFrame(data = y_pred_new.flatten())
ridge_df = pd.concat([df_2019['Player'], pred_df], axis =1)
ridge_df.to_csv('ridge_pred.csv')
ridge_df.sort_values(by=0, ascending=False).head()

,Player,0
33,Kemba Walker\walkeke02,37.910403
7,Kyrie Irving\irvinky01,35.664872
0,Kevin Durant\duranke01,34.205039
4,Kawhi Leonard\leonaka01,33.551864
27,Nikola Vucevic\vucevni01,33.262232


In [25]:
join_df = pd.merge(true_df, ridge_df, on='Player', how='inner')
y_pred = join_df[0]
y_true = join_df['Year_Avg']
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 5.448999


In [29]:
#Load RF model
filename='reg_avg.sav'
avg_model =pickle.load(open(filename, 'rb'))

In [31]:
#Create predictions for 2019 free agents and export into csv using rf model
y_pred_new = avg_model.predict(X_new)
pred_df = pd.DataFrame(data = y_pred_new.flatten())
avg_df = pd.concat([df_2019['Player'], pred_df], axis =1)
avg_df.to_csv('rf_pred.csv')
avg_df.sort_values(by=0, ascending=False).head()

,Player,0
33,Kemba Walker\walkeke02,38.040882
0,Kevin Durant\duranke01,35.544000
7,Kyrie Irving\irvinky01,32.931842
4,Kawhi Leonard\leonaka01,31.851585
27,Nikola Vucevic\vucevni01,29.854435


In [40]:
join_df = pd.merge(true_df, avg_df, on='Player', how='inner')
y_pred = join_df[0]
y_true = join_df['Year_Avg']
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
join_df.to_csv('avg_score.csv')
print("RMSE: %f" % (rmse))

RMSE: 5.103193


In [34]:
#Load stacking model
filename='stack_avg.sav'
stack_model =pickle.load(open(filename, 'rb'))

In [37]:
#Create predictions for 2019 free agents and export into csv using rf model
y_pred_new = stack_model.predict(X_new)
stack_df = pd.DataFrame(data = y_pred_new.flatten())
stack_df = pd.concat([df_2019['Player'], stack_df], axis =1)
stack_df.to_csv('stack_pred.csv')
stack_df.sort_values(by=0, ascending=False).head()

,Player,0
7,Kyrie Irving\irvinky01,36.700554
33,Kemba Walker\walkeke02,36.351983
52,D'Angelo Russell\russeda01,35.675368
4,Kawhi Leonard\leonaka01,35.526109
0,Kevin Durant\duranke01,34.325230


In [39]:
join_df = pd.merge(true_df, stack_df, on='Player', how='inner')
y_pred = join_df[0]
y_true = join_df['Year_Avg']
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 5.424462
